# Analysis of Toronto Neighborhood

## Scrap data from wikipedia:

In [33]:
import requests
from bs4 import BeautifulSoup

In [34]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(page.text, 'html.parser')

In [35]:
import pandas as pd

In [36]:
table_contents=[]
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()



PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [37]:
df.shape

(103, 3)

## Obtaining geolocation data using pgeocode

In [38]:
import pgeocode
from geopy.geocoders import Nominatim

nomi = pgeocode.Nominatim('ca')
geolocator = Nominatim(user_agent="toronto_explorer")

In [96]:
for index, row in df.iterrows():
    location = None

    while(location is None):
        location = nomi.query_postal_code(row['PostalCode'])
        
        if location is not None:
            df.at[index,'Latitude'] = location.latitude
            df.at[index,'Longitude'] = location.longitude               
             
df.head()

PostalCode           Borough                      Neighborhood  Latitude  \
0        M3A        North York                         Parkwoods   43.7545   
1        M4A        North York                  Victoria Village   43.7276   
2        M5A  Downtown Toronto         Regent Park, Harbourfront   43.6555   
3        M6A        North York  Lawrence Manor, Lawrence Heights   43.7223   
4        M7A      Queen's Park     Ontario Provincial Government   43.6641   

   Longitude  
0   -79.3300  
1   -79.3148  
2   -79.3626  
3   -79.4504  
4   -79.3889

## Start of Analysis:

### Step 1 - boroughs and neighborhoods count

In [40]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


### Step 2 - print map of neighborhoods

In [54]:

toronto_location = geolocator.geocode('Toronto, Ontário')
toronto_latitude = location.latitude
toronto_longitude = location.longitude

print(toronto_location,' Lat/Long: ',toronto_latitude,'/',toronto_longitude)

Toronto, Golden Horseshoe, Ontario, Canada  Lat/Long:  43.6256 / -79.5231


In [100]:
# remove rows without geolocation

df.dropna(inplace=True)

In [56]:
import folium # map rendering library

In [101]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Step 4 - Only Boroughs containing Toronto 

In [103]:
tb_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
tb_df.head()

PostalCode           Borough               Neighborhood  Latitude  Longitude
0        M5A  Downtown Toronto  Regent Park, Harbourfront   43.6555   -79.3626
1        M5B  Downtown Toronto   Garden District, Ryerson   43.6572   -79.3783
2        M5C  Downtown Toronto             St. James Town   43.6513   -79.3756
3        M4E      East Toronto                The Beaches   43.6784   -79.2941
4        M5E  Downtown Toronto                Berczy Park   43.6456   -79.3754

### Step 5 - Recount

In [104]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tb_df['Borough'].unique()),
        tb_df.shape[0]
    )
)

The dataframe has 7 boroughs and 39 neighborhoods.


### Step 5 - Reprint map

In [109]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tb_df['Latitude'], tb_df['Longitude'], tb_df['Borough'], tb_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Step 5 - Foursquare exploration and venue collection build

In [106]:
CLIENT_ID = 'MAPXQQR5QDKO0YVCYVFWPZCBQG1UWTLZSQQJZXSGYIRD4VK0' # your Foursquare ID
CLIENT_SECRET = 'I4MAMG2JDMA54ZL21SEPJA15K5QLL4GHMWUQ5W3H3GRFJBOB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MAPXQQR5QDKO0YVCYVFWPZCBQG1UWTLZSQQJZXSGYIRD4VK0
CLIENT_SECRET:I4MAMG2JDMA54ZL21SEPJA15K5QLL4GHMWUQ5W3H3GRFJBOB


Function who gets nearby venues on all location

In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [110]:
toronto_venues = getNearbyVenues(names=tb_df['Neighborhood'],
                                   latitudes=tb_df['Latitude'],
                                   longitudes=tb_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

Size of the resulting dataset

In [111]:
print(toronto_venues.shape)
toronto_venues.head()

(1514, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront                43.6555                -79.3626   
1  Regent Park, Harbourfront                43.6555                -79.3626   
2  Regent Park, Harbourfront                43.6555                -79.3626   
3  Regent Park, Harbourfront                43.6555                -79.3626   
4  Regent Park, Harbourfront                43.6555                -79.3626   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
1        Roselle Desserts       43.653447       -79.362017          Bakery  
2         Berkeley Church       43.655123       -79.365873     Event Space  
3  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
4     Body Blitz Spa East       43.654735       -79.359874             Spa

Venue count by neighborhood

In [112]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            92   
Brockton, Parkdale Village, Exhibition Place                           38   
CN Tower, King and Spadina, Railway Lands, Harb...                     59   
Central Bay Street                                                     60   
Christie                                                               12   
Church and Wellesley                                                   79   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             25   
Davisville North                                                        7   
Dufferin, Dovercourt Village                                           18   
Enclave of M4L                                                         16   
Enclave of M5E                                                        100   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                2   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                       4   
High Park, The Junction South                                           3   
India Bazaar, The Beaches West                                         20   
Kensington Market, Chinatown, Grange Park                              56   
Lawrence Park                                                           2   
Little Portugal, Trinity                                               44   
Moore Park, Summerhill East                                             5   
North Toronto West                                                      4   
Parkdale, Roncesvalles                                                 47   
Regent Park, Harbourfront                                              23   
Richmond, Adelaide, King                                              100   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     51   
St. James Town                                                         92   
St. James Town, Cabbagetown                                            41   
Studio District                                                         7   
Summerhill West, Rathnelly, South Hill, Forest ...                      6   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             8   
The Danforth  East                                                      4   
The Danforth West, Riverdale                                           36   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         26   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             92   
Brockton, Parkdale Village, Exhibition Place                            38   
CN Tower, King and Spadina, Railway Lands, Harb...                      59   
Central Bay Street                                                      60   
Christie                                                                12   
Church and Wellesley                                                    79   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              25   
Davisville North                                                         7   
Dufferin, Dovercourt Village        

How many categories is in it?

In [113]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


### Step 6 - Neighborhood normalization and analisys

In [116]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Adult Boutique  American Restaurant  \
0            0                  0               0                    0   
1            0                  0               0                    0   
2            0                  0               0                    0   
3            0                  0               0                    0   
4            0                  0               0                    0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Athletics & Sports  BBQ Joint  ...  Tibetan Restaurant  Toy / Game Store  \
0                   0          0  ...                   0                 0   
1                   0          0  ...                   0                 0   
2                   0          0  ...                   0                 0   
3                   0          0  ...                   0                 0   
4                   0          0  ...                   0                 0   

   Trail  Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0      0              0                              0                 0   
1      0              0                              0                 0   
2      0              0                              0                 0   
3      0              0                              0                 0   
4      0              0                              0                 0   

   Vietnamese Restaurant  Whisky Bar  Wine Bar  Wings Joint  
0                      0           0         0            0  
1                      0           0         0            0  
2                      0           0         0            0  
3                      0           0         0            0  
4                      0           0         0            0  

[5 rows x 217 columns]

In [118]:
toronto_onehot.shape

(1514, 217)

Group neighborhood data

In [120]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.010870   
1       Brockton, Parkdale Village, Exhibition Place     0.000000   
2  CN Tower, King and Spadina, Railway Lands, Har...     0.016949   
3                                 Central Bay Street     0.000000   
4                                           Christie     0.000000   

   Accessories Store  Adult Boutique  American Restaurant  Art Gallery  \
0           0.000000             0.0              0.01087     0.021739   
1           0.026316             0.0              0.00000     0.026316   
2           0.000000             0.0              0.00000     0.000000   
3           0.000000             0.0              0.00000     0.000000   
4           0.000000             0.0              0.00000     0.000000   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  ...  \
0    0.000000             0.000000          0.000000            0.000000  ...   
1    0.000000             0.026316          0.000000            0.000000  ...   
2    0.000000             0.000000          0.016949            0.000000  ...   
3    0.016667             0.000000          0.000000            0.000000  ...   
4    0.000000             0.000000          0.000000            0.083333  ...   

   Tibetan Restaurant  Toy / Game Store  Trail  Train Station  \
0                 0.0               0.0    0.0            0.0   
1                 0.0               0.0    0.0            0.0   
2                 0.0               0.0    0.0            0.0   
3                 0.0               0.0    0.0            0.0   
4                 0.0               0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                        0.01087          0.000000               0.000000   
1                        0.00000          0.000000               0.000000   
2                        0.00000          0.000000               0.000000   
3                        0.00000          0.016667               0.016667   
4                        0.00000          0.000000               0.000000   

   Whisky Bar  Wine Bar  Wings Joint  
0         0.0  0.000000     0.000000  
1         0.0  0.000000     0.000000  
2         0.0  0.000000     0.016949  
3         0.0  0.016667     0.000000  
4         0.0  0.000000     0.000000  

[5 rows x 217 columns]

In [121]:
toronto_grouped.shape

(39, 217)

Top 5 common venus by neighborhood

In [122]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1               Hotel  0.05
2              Bakery  0.05
3                Café  0.04
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2  Thrift / Vintage Store  0.05
3          Breakfast Spot  0.05
4               Gift Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3                 Bar  0.05
4           Speakeasy  0.03


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.03
2         Pizza Place  0.03
3     Bubble Tea Shop  0.03
4      Sandwich Place  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1       

### Step 7 collect and compare top 10 venues

In [125]:
import numpy as np

In [123]:
# common venus in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
# dataframe with top venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  CN Tower, King and Spadina, Railway Lands, Har...    Italian Restaurant   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie         Grocery Store   

  2nd Most Common Venue   3rd Most Common Venue 4th Most Common Venue  \
0                 Hotel                  Bakery                  Café   
1           Coffee Shop  Thrift / Vintage Store        Breakfast Spot   
2           Coffee Shop                    Café                   Bar   
3    Italian Restaurant             Pizza Place       Bubble Tea Shop   
4                  Café      Athletics & Sports           Candy Store   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Seafood Restaurant              Beer Bar            Restaurant   
1             Gift Shop           Supermarket              Boutique   
2             Speakeasy     French Restaurant         Grocery Store   
3        Sandwich Place            Restaurant      Sushi Restaurant   
4           Coffee Shop            Baby Store            Playground   

  8th Most Common Venue      9th Most Common Venue      10th Most Common Venue  
0          Cocktail Bar        Japanese Restaurant          Italian Restaurant  
1            Restaurant                    Brewery                         Gym  
2            Restaurant       Gym / Fitness Center                        Park  
3                  Café  Middle Eastern Restaurant  Modern European Restaurant  
4                  Park       Other Great Outdoors                 Music Store

### Step 8 - Clustering

In [129]:
from sklearn.cluster import KMeans

In [173]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [174]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tb_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough               Neighborhood  Latitude  \
0        M5A  Downtown Toronto  Regent Park, Harbourfront   43.6555   
1        M5B  Downtown Toronto   Garden District, Ryerson   43.6572   
2        M5C  Downtown Toronto             St. James Town   43.6513   
3        M4E      East Toronto                The Beaches   43.6784   
4        M5E  Downtown Toronto                Berczy Park   43.6456   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0   -79.3626               2           Coffee Shop        Breakfast Spot   
1   -79.3783               2           Coffee Shop        Clothing Store   
2   -79.3756               2                  Café           Coffee Shop   
3   -79.2941               2                   Pub                 Trail   
4   -79.3754               2           Coffee Shop                 Hotel   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Yoga Studio       Thai Restaurant    Italian Restaurant   
1        Sandwich Place                  Café                 Hotel   
2    Seafood Restaurant             Gastropub    Italian Restaurant   
3           Cheese Shop     Health Food Store                Bakery   
4                Bakery                  Café    Seafood Restaurant   

       6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0       Gym / Fitness Center            Food Truck           Event Space   
1  Middle Eastern Restaurant           Pizza Place        Cosmetics Shop   
2               Cocktail Bar                Bakery            Restaurant   
3                  Gastropub   Monument / Landmark                Museum   
4                   Beer Bar            Restaurant          Cocktail Bar   

  9th Most Common Venue 10th Most Common Venue  
0     Electronics Store    Distribution Center  
1   Japanese Restaurant     Italian Restaurant  
2              Beer Bar         Farmers Market  
3         Movie Theater    Moroccan Restaurant  
4   Japanese Restaurant     Italian Restaurant

In [175]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [176]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 9 - Cluster analysis

In [177]:
def show_cluster(cluster):
    return toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [178]:
show_cluster(0)

Borough  Cluster Labels 1st Most Common Venue  \
21  Central Toronto               0                 Trail   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21                Lawyer           Yoga Studio           Music Store   

   5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
21           Men's Store    Mexican Restaurant  Middle Eastern Restaurant   

   8th Most Common Venue       9th Most Common Venue  \
21    Miscellaneous Shop  Modern European Restaurant   

             10th Most Common Venue  
21  Molecular Gastronomy Restaurant

In [179]:
show_cluster(1)

Borough  Cluster Labels 1st Most Common Venue  \
9   East York/East Toronto               1                  Park   
10        Downtown Toronto               1           Music Venue   
18         Central Toronto               1    Photography Studio   
22            West Toronto               1         Bowling Alley   

   2nd Most Common Venue                     3rd Most Common Venue  \
9      Convenience Store                              Intersection   
10                  Park                                      Café   
18                  Park                               Music Venue   
22                  Park  Residential Building (Apartment / Condo)   

   4th Most Common Venue    5th Most Common Venue      6th Most Common Venue  \
9            Yoga Studio  New American Restaurant  Middle Eastern Restaurant   
10       Harbor / Marina              Yoga Studio         Mexican Restaurant   
18           Men's Store       Mexican Restaurant  Middle Eastern Restaurant   
22           Yoga Studio              Music Venue         Mexican Restaurant   

        7th Most Common Venue       8th Most Common Venue  \
9          Miscellaneous Shop  Modern European Restaurant   
10  Middle Eastern Restaurant          Miscellaneous Shop   
18         Miscellaneous Shop  Modern European Restaurant   
22  Middle Eastern Restaurant          Miscellaneous Shop   

              9th Most Common Venue           10th Most Common Venue  
9   Molecular Gastronomy Restaurant              Monument / Landmark  
10       Modern European Restaurant  Molecular Gastronomy Restaurant  
18  Molecular Gastronomy Restaurant              Monument / Landmark  
22       Modern European Restaurant  Molecular Gastronomy Restaurant

In [180]:
show_cluster(2)

Borough  Cluster Labels 1st Most Common Venue  \
0         Downtown Toronto               2           Coffee Shop   
1         Downtown Toronto               2           Coffee Shop   
2         Downtown Toronto               2                  Café   
3             East Toronto               2                   Pub   
4         Downtown Toronto               2           Coffee Shop   
5         Downtown Toronto               2           Coffee Shop   
6         Downtown Toronto               2         Grocery Store   
7         Downtown Toronto               2                  Café   
8             West Toronto               2                  Park   
11            West Toronto               2                   Bar   
12            East Toronto               2      Greek Restaurant   
13        Downtown Toronto               2           Coffee Shop   
14            West Toronto               2                  Café   
15            East Toronto               2            Restaurant   
16        Downtown Toronto               2           Coffee Shop   
17            East Toronto               2        Baseball Field   
20         Central Toronto               2               Dog Run   
24         Central Toronto               2        Sandwich Place   
25            West Toronto               2                Bakery   
26         Central Toronto               2           Coffee Shop   
27        Downtown Toronto               2                  Café   
28            West Toronto               2                  Café   
29         Central Toronto               2       Thai Restaurant   
30        Downtown Toronto               2                  Café   
31         Central Toronto               2           Coffee Shop   
32        Downtown Toronto               2    Italian Restaurant   
34  Downtown Toronto Stn A               2           Coffee Shop   
35        Downtown Toronto               2           Coffee Shop   
36        Downtown Toronto               2           Coffee Shop   
37        Downtown Toronto               2   Japanese Restaurant   
38   East Toronto Business               2            Restaurant   

            2nd Most Common Venue        3rd Most Common Venue  \
0                  Breakfast Spot                  Yoga Studio   
1                  Clothing Store               Sandwich Place   
2                     Coffee Shop           Seafood Restaurant   
3                           Trail                  Cheese Shop   
4                           Hotel                       Bakery   
5              Italian Restaurant                  Pizza Place   
6                            Café           Athletics & Sports   
7                     Coffee Shop                        Hotel   
8                          Bakery                  Post Office   
11                     Restaurant                  Coffee Shop   
12                     Restaurant               Ice Cream Shop   
13                          Hotel                         Café   
14                    Coffee Shop       Thrift / Vintage Store   
15           Fast Food Restaurant                          Gym   
16                           Café                        Hotel   
17                Coworking Space                          Gym   
20               Department Store                   Playground   
24                           Café            Indian Restaurant   
25                    Coffee Shop  Eastern European Restaurant   
26                           Café                 Dessert Shop   
27            Japanese Restaurant                    Bookstore   
28                    Coffee Shop                  Pizza Place   
29                           Park                        Trail   
30  Vegetarian / Vegan Restaurant                       Bakery   
31             Light Rail Station                 Liquor Store   
32                    Coffee Shop                         Café   
34                     Restaurant                        Hot

In [181]:
show_cluster(3)

Borough  Cluster Labels 1st Most Common Venue  \
23   Central Toronto               3            Playground   
33  Downtown Toronto               3            Playground   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23                  Park              Gym Pool                Garden   
33                  Park           Candy Store         Grocery Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23           Yoga Studio           Music Store    Mexican Restaurant   
33           Yoga Studio           Music Store    Mexican Restaurant   

        8th Most Common Venue 9th Most Common Venue  \
23  Middle Eastern Restaurant    Miscellaneous Shop   
33  Middle Eastern Restaurant    Miscellaneous Shop   

        10th Most Common Venue  
23  Modern European Restaurant  
33  Modern European Restaurant

In [182]:
show_cluster(4)

Borough  Cluster Labels 1st Most Common Venue  \
19  Central Toronto               4          Home Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
19           Music Venue           Men's Store    Mexican Restaurant   

        5th Most Common Venue 6th Most Common Venue  \
19  Middle Eastern Restaurant    Miscellaneous Shop   

         7th Most Common Venue            8th Most Common Venue  \
19  Modern European Restaurant  Molecular Gastronomy Restaurant   

   9th Most Common Venue 10th Most Common Venue  
19   Monument / Landmark    Moroccan Restaurant

## Conclusion

- Cluster 0: very different of the others because your main venue category is trail, because the presence of the forest
- Cluster 1: parks, music and photography
- Cluster 2: food and hotels
- Cluster 3: playgrounds and parks
- Cluster 4: Home Service
